<a class="anchor" id="top"></a>

# Content Based Recommendation System

This notebook creates a content-based recommendation system to calculate product similarity using the cosine-similarity method. Our recommendation system will use these cosine-similarity values to identify the most similar marker products, which in turn will identify the best question to ask at PoS.

## Table of Contents

[Step 1: Import Required Packages](#step-1) <br>
[Step 2: Read Datasets](#step-2) <br>
[Step 3: Data Preparation for model](#step-3) <br>
[Step 4: Build Content Based Recommendation System](#step-4) <br>

<a class="anchor" id="step-1"></a>

## Step 1: Import Required Packages

In [5]:
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='76c479a0-4c1c-44db-b8a2-becdfbbf3281', project_access_token='p-df8a7e5431c2264f03914c5f141281df62d590ef')

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from io import BytesIO
# import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from scipy import sparse
# from pprint import pprint

pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199

[nltk_data] Downloading package stopwords to /home/wsuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Required Functions

In [6]:
# These are the function that are used later in the notebook to clean the description of the products.
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

[Back to the Top](#top)

<a class="anchor" id="step-2"></a>

## Step 2: Read Datasets



In [7]:
# Read IBM_WATSON_ARTICLE_FULL dataset, include necessary columns to be extracted
df_article= pd.read_csv(project.get_file("IBM_WATSON_ARTICLE_FULL.csv"), usecols=['ARTICLE_NO','ARTICLE_DESC'])

print('df_article shape', df_article.shape)
print('--------------------------------------')
df_article.head(2)

df_article shape (856321, 2)
--------------------------------------


,ARTICLE_NO,ARTICLE_DESC
0,748043,NFL - ST. LOUIS RAMS GET A GRIP 2 PACK
1,748046,KDS RENEGADE 400 BOOT TAN CAMO 13


In [8]:
# Read ExtraSamples_Questions dataset, include necessary columns to be extracted
df_questions= pd.read_csv(project.get_file("QUESTION_MARKERS.csv"), usecols=['Question','ARTICLE_NO','ARTICLE_DESC'])

print('df_questions shape', df_questions.shape)
print('--------------------------------------')
df_questions.head(2)

df_questions shape (40, 3)
--------------------------------------


,Question,ARTICLE_NO,ARTICLE_DESC
0,Would you like to sign up for a pet food subscription?,1555466,ONE 4LB SM BITE BEEF/RICE
1,Would you like to sign up for a pet food subscription?,1397138,IAMS 4.4# NTRLS SENSITIVE


In [9]:
# Read click purchase dataset and assign "Purchase" name to the new column of ClickType.
df_click_purchases = pd.read_csv(project.get_file("IBM_WATSON_CLICK_PURCHASE.csv"))

print('df_click_purchases shape', df_click_purchases.shape)
print('--------------------------------------')
df_click_purchases.head(2)

df_click_purchases shape (229598, 4)
--------------------------------------


,VISITOR_CLICK_ID,TIME_DIM_KEY,ARTICLE_NO,SAP_BP_ID
0,3398575622603852750_2390095112132868450_6_1616511665_1616511932_16,20210323,705057,164093948
1,5561444425824395160_4824659609152105886_2_1608568403_1608569227_38,20201221,389237,76116948


In [10]:
# Read the VW_SALES dataset, include necessary columns to be extracted
df_sales = pd.read_csv(project.get_file("IBM_WATSON_VW_SALES.csv"), usecols=['TIME_DIM_KEY', 'ARTICLE_NO', 'SAP_BP_ID'])

print('df_sales shape', df_sales.shape)
print('--------------------------------------')
df_sales.head(2)

df_sales shape (21101188, 3)
--------------------------------------


,ARTICLE_NO,TIME_DIM_KEY,SAP_BP_ID
0,308453,20210727,"417,504,379.00"
1,139852,20210727,"417,504,379.00"


In [11]:
# Read original Customer dataset
df_customers = pd.read_csv(project.get_file('IBM_WATSON_SALES_SAP_BP_ID.csv'))

# About 1/3 of the customers are in the loyalty program. We will only focus on those customers, and remove the rest from analysis.
df_customers = df_customers[df_customers['LOYALTY_FLAG']=='Y']

# Now, we need to see how many of the loyalty customers have clean data. We will only focus on those customers.
df_customers = df_customers[df_customers['CLEAN_FLAG']=='Y']

print('df_customers shape', df_customers.shape)
print('--------------------------------------')
df_customers.head(2)

df_customers shape (335051, 3)
--------------------------------------


,SAP_BP_ID,LOYALTY_FLAG,CLEAN_FLAG
0,167709999,Y,Y
2,111792373,Y,Y


In [12]:
# merge the customer data with click purchase data and include only necessary columns
df_click_purchases_subset = pd.merge(df_customers, df_click_purchases, how="inner", on="SAP_BP_ID")
df_click_purchases_subset = df_click_purchases_subset[['SAP_BP_ID','TIME_DIM_KEY','ARTICLE_NO']]

print('df_click_purchases_subset shape', df_click_purchases_subset.shape)
print('--------------------------------------')
df_click_purchases_subset.head(2)

df_click_purchases_subset shape (208238, 3)
--------------------------------------


,SAP_BP_ID,TIME_DIM_KEY,ARTICLE_NO
0,401162293,20210106,123047
1,72643124,20210331,623840


In [13]:
# merge the customer data with click purchase data and include only necessary columns
df_sales_subset = pd.merge(df_customers, df_sales, how="inner", on="SAP_BP_ID")
df_sales_subset = df_sales_subset[['SAP_BP_ID','TIME_DIM_KEY','ARTICLE_NO']]

print('df_sales_subset shape', df_sales_subset.shape)
print('--------------------------------------')
df_sales_subset.head(2)

df_sales_subset shape (13117108, 3)
--------------------------------------


,SAP_BP_ID,TIME_DIM_KEY,ARTICLE_NO
0,167709999,20170828,332069
1,167709999,20170520,841026


In [14]:
# Make transactions dataframe by concatenating the purchase click dataframe with the sales data. Cast TIME_DIM_KEY to datetime type.
df_transactions = pd.concat([df_sales_subset, df_click_purchases_subset], axis=0)
df_transactions['TIME_DIM_KEY'] = pd.to_datetime(df_transactions['TIME_DIM_KEY'], format='%Y%m%d')

print('df_transactions shape', df_transactions.shape)
print('--------------------------------------')
df_transactions.head(2)

df_transactions shape (13325346, 3)
--------------------------------------


,SAP_BP_ID,TIME_DIM_KEY,ARTICLE_NO
0,167709999,2017-08-28,332069
1,167709999,2017-05-20,841026


<a class="anchor" id="step-3"></a>

## Step3: Data Preparation for model

In [15]:
# We will only keep transactional data from the past two years
df_transactions['Year'] = pd.DatetimeIndex(df_transactions['TIME_DIM_KEY']).year
df_transactions = df_transactions.loc[(df_transactions['Year'] == 2020) | (df_transactions['Year'] == 2021)]

# Merge the article description with transactional data
df_transactions = pd.merge(df_transactions, df_article, on=["ARTICLE_NO"], how="inner")

print('df_transactions shape', df_transactions.shape)
print('--------------------------------------')
df_transactions.head(2)

df_transactions shape (5602878, 5)
--------------------------------------


,SAP_BP_ID,TIME_DIM_KEY,ARTICLE_NO,Year,ARTICLE_DESC
0,111792373,2021-05-06,209612,2021,FLIPPIN FISH 11.5IN
1,182786234,2021-04-28,209612,2021,FLIPPIN FISH 11.5IN


In [16]:
# generate dataframe of products and their descriptions, sorted by the highest transactional volume
df_products = df_transactions[['ARTICLE_NO', 'ARTICLE_DESC', 'SAP_BP_ID']]
df_products = df_products.groupby(['ARTICLE_NO', 'ARTICLE_DESC']).count().sort_values(by='SAP_BP_ID', ascending=False).reset_index()[['ARTICLE_NO', 'ARTICLE_DESC']]

# add the marker products to this dataframe
df_products = pd.concat([df_questions[['ARTICLE_NO', 'ARTICLE_DESC']], df_products], axis=0)

print('df_products shape', df_products.shape)
print('--------------------------------------')
df_products.head(2)

df_products shape (55571, 2)
--------------------------------------


,ARTICLE_NO,ARTICLE_DESC
0,1555466,ONE 4LB SM BITE BEEF/RICE
1,1397138,IAMS 4.4# NTRLS SENSITIVE


In [17]:
# Use the required functions that are defined above to clean the description of the products, to remove the punctuation and some unncessary symbols and add a new column to dataframe as clean description.
df_products['cleaned_DESC'] = df_products['ARTICLE_DESC'].apply(_removeNonAscii)
df_products['cleaned_DESC'] = df_products.cleaned_DESC.apply(func = remove_stop_words)
df_products['cleaned_DESC'] = df_products.cleaned_DESC.apply(func=remove_punctuation)
df_products['cleaned_DESC'] = df_products.cleaned_DESC.apply(func=remove_html)

In [18]:
# project.save_data('df_products_cos_sim.csv', df_product_sorted.to_csv(index=False), overwrite=True)

<a class="anchor" id="step-4"></a>

## Step 4: Build Content Based Recommendation System

Cosine similarity measures the similarity between two vectors of an inner product space. It is measured by the cosine of the angle between two vectors and determines whether two vectors are pointing in roughly the same direction. It is often used to measure document similarity in text analysis.

In [19]:
# Here, we use TfidfVectorizer to transform text to feature vectors that can be used as inputs to the cosine-similarity estimator.
vectorizer = TfidfVectorizer(analyzer='word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = vectorizer.fit_transform(df_products['cleaned_DESC'])
tfidf_matrix.shape

(55571, 126523)

In [20]:
# Compute the cosine similarity matrix for tfidf_matrix using linear_kernel
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
# We compress the cosine similarity matrix into a CSR matrix to save space
compressed = sparse.csr_matrix(cosine_similarity)
compressed

In [21]:
# Save the cosine similarity CSR to file

filename = 'sparse_mat.npz'
sparse.save_npz('sparse_mat.npz', compressed)
with open('sparse_mat.npz', 'rb') as f:
    _ = project.save_data(filename, f, overwrite=True)